<a href="https://colab.research.google.com/github/arkoday/Autocorrect/blob/main/Autocorrect_Spelling_Checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Steps for implementing a very simple autocorrect system without using any deep neural network. Some times a very simple easy solution outperforms a complex one!!!

1. Identify the mispelled word. A word is mispelled if it is not found in the vocabulary of the corpus of text the autocorrect system is working with.
2. Perform edit operations in order to find strings that are n edit distance away from the mispelled word.
3. Filter our the words to retain only the ones found in the vocabulary
4. Order words based on probability. The probabilities of a word is calculated on the following formula : p(w) = C(w)/V, where 
p(w) is the probability of a word w,
C(w) is the number of times a word w is appered in the training corpus,
v is the total no of words in the corpus
5. Choose the most likely word

In [62]:
import re
import string
from collections import Counter
import numpy as np




class Autocorrect():


  def __init__(self,corpus_file):
    with open(corpus_file , 'r') as file:
      lines = file.readlines()
      words=[]
      for line in lines:
        words+= re.findall(r'\w+',line.lower()) ## total no of words present in my corpus
        

    self.unq_words = set(words)   ## Evaluating only the unique words present in my corpus
    words_counter = Counter(words)  ## Calculating frequency of each word present in the corpus
    total_words=float(sum(words_counter.values()))
    self.word_probability = { key:words_counter[key] / total_words  for key in words_counter.keys()} ## probability of each word based on its occurrence in the corpus



  def edit_one_distance(self,word):
    '''
    performing edit operations such as split,insert,delete,swap and replace in order to find list of words that are One-edit-distance 
    away from a given word.

    '''
    eng_letters = string.ascii_lowercase
    split_word = [(word[:i],word[i:]) for i in range(len(word)+1)]
    insert_word = [i+c+r for i,r in split_word if r for c in eng_letters]
    delete_word = [i+r[1:] for i,r in split_word if r]
    swap_word = [i+r[1]+r[0]+r[2:] for i,r in split_word if len(r)>1]
    replace_word = [i+c+r[1:] for i,r in split_word for c in eng_letters]

    return set(insert_word + delete_word + swap_word + replace_word)

  def edit_two_distance(self,word):
    '''
    With the help of edit_one_distance function and list comprehension we can easily find out words that are Two-edit-distance away from a given word

    '''

    return set(e2 for e1 in edit_one_distance(word) for e2 in edit_one_distance(e1))


  def corrections(self, word):

    '''
    Here we are collecting all possible suggestions that are max Two-edit_distance away from a given word and removing the ones which are not 
    present in my corpus data.
    The words which are valid and are max Two-edit-distance away from a given mispelled word is stored in best_guesses.
    and we are suggesting valid words based on their probability

    '''
    suggestions = self.edit_one_distance(word) or self.edit_two_distance(word) or [word]
    best_guesses = [w for w in suggestions if w in self.unq_words]
    return sorted([(w , self.word_probability[w]) for w in best_guesses], key= lambda tup:tup[1],reverse=True)










    

In [63]:
checker = Autocorrect("file_path")
checker.corrections('loue')

[('love', 0.001193993549118186),
 ('loud', 7.186072286359452e-05),
 ('lose', 4.6985857256965655e-05),
 ('lore', 2.763873956292097e-06)]